### Minimization of polynomial objective

In [1]:
using Symbolics
using LinearAlgebra
using DynamicPolynomials

In [12]:
Threads.nthreads()

1

In [11]:
import Ipopt

Declare functions to constract Pade objective

In [3]:
function frobenius_norm2(m)
    return tr(m * m')
end

function lindblad_rhs(rho, H, A)
    """
    Right hand side of the Lindblad master equation
    """
    return -1im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
    
end

function pade_obj(ρ, t, H, A)
    
    obj = 0
    for i in 2:size(ρ,3)
        obj += frobenius_norm2(
            ρ[:, :, i] - ρ[:, :, 1] 
            - (t[i]-t[i-1])*lindblad_rhs((ρ[:, :, i]+ρ[:, :, i-1])/2, H, A)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

pade_obj (generic function with 1 method)

Construct sample objective

In [8]:
@polyvar x[1:4]
H = [ 1.0 * x[1]              x[3] + im * x[4]
      x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]

ρ = [ρ1;;; ρ2;;; ρ3;;; ρ4]
t = [0.0, 0.1, 0.2, 0.3]

obj = pade_obj(ρ, t, H, A);

Use Optim.jl package to perfrom optimixation

In [10]:
res = Optim.optimize(pade_obj, variables(obj))

LoadError: MethodError: no method matching real(::Polynomial{true, Int64})
[0mClosest candidates are:
[0m  real([91m::Diagonal[39m) at C:\Users\Zakhar\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\LinearAlgebra\src\diagonal.jl:139
[0m  real([91m::UnitUpperTriangular{var"#s858", S} where {var"#s858"<:Real, S<:AbstractMatrix{var"#s858"}}[39m) at C:\Users\Zakhar\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\LinearAlgebra\src\triangular.jl:48
[0m  real([91m::UnitUpperTriangular{var"#s857", S} where {var"#s857"<:Complex, S<:AbstractMatrix{var"#s857"}}[39m) at C:\Users\Zakhar\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\LinearAlgebra\src\triangular.jl:49
[0m  ...

In [6]:
Optim.minimizer(res)

LoadError: UndefVarError: res not defined

In [7]:
Optim.minimum(res)(res)

LoadError: UndefVarError: res not defined